In [1]:
# Import Libraries
%matplotlib inline
import re
import matplotlib.pyplot as plt
import numpy as np
import Bio.SeqIO as SeqIO
import time
from sklearn.cluster import KMeans, k_means

In [34]:
# Extract Data
j = 0
dict = []
indonesian_virus = []
for i, record in enumerate(SeqIO.parse("sequence.gb", "genbank")):
    if all (key in record.features[0].qualifiers for key in ('host','country','collection_date')):
        j+=1
        
        host = record.features[0].qualifiers['host'][0]
        host = host.split(';')[0]
        
        location = record.features[0].qualifiers['country'][0]
        location = location.split(':')[0]
        
        year = record.features[0].qualifiers['collection_date'][0]
        year = year.split('-')[-1]
        
        row = {
            'id'          :record.id,
            'name'        :record.name,
            'description' :record.description,
            'host'        :host,
            'location'    :location,
            'year'        :year,
            'seq'         :record.seq,
            'url'         :'http://www.ncbi.nlm.nih.gov/nuccore/'+record.id
        }
        
        if location == 'Indonesia':
            indonesian_virus.append(row)
            
        else:
            dict.append(row)

In [ ]:
for row in dict:
    print (row['id'] + ' ' + row['url'])

In [4]:
def match_score(A,B):
    if A == B: return 1
    else: return -1

In [5]:
def print_table(S,header):
    print(header)
    for i in range(0,len(S)):
        print(S[i])
    print('\n')

In [51]:
# Global Alignment
def complete_global_alignment(v,w):
    m = len(v)
    n = len(w)

    # Backtrack enum
    R_UP = 1
    R_LEFT = 2
    R_DIAG = 3

    d = -1
    S = [[0 for x in range(len(v)+1)] for y in range(len(w)+1)]
    B = [[0 for x in range(len(v)+1)] for y in range(len(w)+1)]

    for i in range(1,len(w)+1):
        S[i][0] = d*i
        B[i][0] = R_UP
    for j in range(1,len(v)+1):
        S[0][j] = d*j
        B[0][j] = R_LEFT
    for i in range(1,len(w)+1):
        for j in range(1,len(v)+1):
            match_score = 1 if v[j-1] == w[i-1] else -1
            Match = S[i-1][j-1] + match_score
            Insert = S[i-1][j] + d
            Delete = S[i][j-1] + d
            S[i][j] = max(Match, Insert, Delete)
            if S[i][j] == Match:
                B[i][j] = R_DIAG
            elif S[i][j] == Insert:
                B[i][j] = R_UP
            elif S[i][j] == Delete:
                B[i][j] = R_LEFT
    
    # Print resulting table
    #print_table(S,'Resulting Table')
    
    # Print backtrack matrix
    #print_table(B,'Backtrack Matrix')
    
    # Print resulting string
    #print('Backtrack Process')
    vr = ""
    wr = ""
    i = n
    j = m
    k = 0
    while B[i][j] != 0:
        if B[i][j] == R_DIAG:
            vr = v[j-1] + vr
            wr = w[i-1] + wr
            #print(i,j,'DIAG')
            i = i-1
            j = j-1
        elif B[i][j] == R_LEFT:
            wr = '-' + wr
            vr = v[j-1] + vr
            #print(i,j,'LEFT')
            j = j-1
        elif B[i][j] == R_UP:
            vr = '-' + vr
            wr = w[i-1] + wr
            #print(i,j,'UP')
            i = i-1

    #print('\n')
    
    #print('Global Alignment')
    #print('Length v:')
    #print(len(vr))
    #print('Length w:')
    #print(len(wr))
    #print(vr)
    #print(wr)
    #print('Score:')
    #print(S[len(w)][len(v)])
    return S[len(w)][len(v)]

In [3]:
# Global Alignment
def global_alignment(v,w):
    m = len(v)
    n = len(w)

    d = -1
    S = [[0 for x in range(len(v)+1)] for y in range(len(w)+1)]

    for i in range(1,len(w)+1):
        S[i][0] = d*i
    for j in range(1,len(v)+1):
        S[0][j] = d*j
    for i in range(1,len(w)+1):
        for j in range(1,len(v)+1):
            match_score = 1 if v[j-1] == w[i-1] else -1
            Match = S[i-1][j-1] + match_score
            Insert = S[i-1][j] + d
            Delete = S[i][j-1] + d
            S[i][j] = max(Match, Insert, Delete)
                
    return S[len(w)][len(v)]

In [49]:
def complete_local_alignment(v,w):
    m = len(v)
    n = len(w)

    # Backtrack enum
    R_UP = 1
    R_LEFT = 2
    R_DIAG = 3

    d = -1
    max_val = -1
    max_row = 0
    max_col = 0
    S = [[0 for x in range(len(v)+1)] for y in range(len(w)+1)]
    B = [[0 for x in range(len(v)+1)] for y in range(len(w)+1)]

    for i in range(1,len(w)+1):
        for j in range(1,len(v)+1):
            match_score = 1 if v[j-1] == w[i-1] else -1
            Match = S[i-1][j-1] + match_score
            Insert = S[i-1][j] + d
            Delete = S[i][j-1] + d
            S[i][j] = max(Match, Insert, Delete, 0)
            if S[i][j] > max_val:
                max_val = S[i][j]
                max_row = i
                max_col = j
            if S[i][j] == 0:
                continue
            elif S[i][j] == Match:
                B[i][j] = R_DIAG
            elif S[i][j] == Insert:
                B[i][j] = R_UP
            elif S[i][j] == Delete:
                B[i][j] = R_LEFT

    # Print resulting table
    #print_table(S,'Resulting Table')
          
    # Print backtrack matrix
    #print_table(B,'Backtrack Matrix')
    
    # Print resulting string
    #print('Backtrack Process')
    vr = ""
    wr = ""
    i = max_row
    j = max_col
    #print('Max Row',max_row,'Max Col',max_col)
    while B[i][j] > 0:
        if B[i][j] == R_DIAG:
            vr = v[j-1] + vr
            wr = w[i-1] + wr
            #print(i,j,'DIAG')
            i = i-1
            j = j-1
        elif B[i][j] == R_LEFT:
            #wr = '-' + wr
            vr = v[j-1] + vr
            #print(i,j,'LEFT')
            j = j-1
        elif B[i][j] == R_UP:
            #vr = '-' + vr
            wr = w[i-1] + wr
            #print(i,j,'UP')
            i = i-1

    #print('\n')
    
    #print('Local Alignment')
    #print('Length v:')
    #print(len(vr))
    #print('Length w:')
    #print(len(wr))
    #print(vr)
    #print(wr)
    #vx = vr
    #wx = wr
    #print('Score:')
    #print(max_val)
    #return vr,wr
    return max_val

In [4]:
def local_alignment(v,w):
    m = len(v)
    n = len(w)

    d = -1
    max_val = -1
    max_row = 0
    max_col = 0
    S = [[0 for x in range(len(v)+1)] for y in range(len(w)+1)]

    for i in range(1,len(w)+1):
        for j in range(1,len(v)+1):
            match_score = 1 if v[j-1] == w[i-1] else -1
            Match = S[i-1][j-1] + match_score
            Insert = S[i-1][j] + d
            Delete = S[i][j-1] + d
            S[i][j] = max(Match, Insert, Delete, 0)
            if S[i][j] > max_val:
                max_val = S[i][j]

    return max_val

In [11]:
#get centroid
seq_len_kmeans = [[len(row['seq'])] for row in dict]
seq_centroid, seq_cluster, seq_cluster_error = k_means(seq_len_kmeans, n_clusters=2)
seq_centroid

array([[   904.32666667],
       [ 10614.50588235]])

In [12]:
#get threshold: the average of centroids
threshold = (seq_centroid[0] + seq_centroid[1])/2
threshold

array([ 5759.41627451])

In [35]:
len(indonesian_virus[0]['seq'])

1148

In [36]:
len(dict[2]['seq'])

10735

In [10]:
#individual check
start_time = time.time() # execution time
score = local_alignment(indonesian_virus[0]['seq'],dict[2]['seq'])
print(score)
print("Time elapsed: %s seconds" % (time.time() - start_time))

556
Time elapsed: 21.93324899673462 seconds


In [13]:
#individual check
start_time = time.time() # execution time
v = str(indonesian_virus[0]['seq'])
w = str(dict[2]['seq'])
score = local_alignment(v,w)
print(score)
print("Time elapsed: %s seconds" % (time.time() - start_time))

556
Time elapsed: 13.243183135986328 seconds


In [53]:
#individual check
start_time = time.time() # execution time
v = str(indonesian_virus[0]['seq'])
w = str(dict[2]['seq'])
score = local_alignment(v,w)
print(score)
print("Time elapsed: %s seconds" % (time.time() - start_time))

556
Time elapsed: 17.191015005111694 seconds


In [52]:
#group check
start_time = time.time() # execution time
score = []
i = 1
for ind in indonesian_virus:
    for row in dict:
        print(i)
        virus_time = time.time();
        if ind['id'] == row['id']:
            continue
        print(ind['id'] , ' vs ' , row['id'])
        i += 1
        s = ""
        if len(row['seq']) > threshold:
            print('Local')
            s = local_alignment(ind['seq'],row['seq'])
        else:
            print('Global')
            s = global_alignment(ind['seq'],row['seq'])
        score.append(s)
        print('Score: ' , s)
        print("Time elapsed: %s seconds" % (time.time() - virus_time))
        if i > 10: #Flow control to test
            break
print("Total time elapsed: %s seconds" % (time.time() - start_time))

1
KU179098.1  vs  KX101066.1
Local
Score:  290
Time elapsed: 21.58214497566223 seconds
2
KU179098.1  vs  KX101065.1
Local
Score:  180
Time elapsed: 21.251593112945557 seconds
3
KU179098.1  vs  KX101064.1
Local
Score:  556
Time elapsed: 22.33268404006958 seconds
4
KU179098.1  vs  KX101063.1
Local
Score:  135
Time elapsed: 21.783394813537598 seconds
5
KU179098.1  vs  KX101062.1
Local
Score:  178
Time elapsed: 21.886754989624023 seconds
6
KU179098.1  vs  KX101061.1
Local
Score:  694
Time elapsed: 23.080465078353882 seconds
7
KU179098.1  vs  KX101060.1
Local
Score:  924
Time elapsed: 21.181591987609863 seconds
8
KU179098.1  vs  KX377337.1
Local
Score:  1126
Time elapsed: 24.648312091827393 seconds
9
KU179098.1  vs  KX377336.1
Local
Score:  1035
Time elapsed: 26.91932702064514 seconds
10
KU179098.1  vs  KX377335.1
Local
Score:  888
Time elapsed: 27.83112382888794 seconds
11
KF258813.1  vs  KX101066.1
Local
Score:  80
Time elapsed: 8.472402095794678 seconds
Total time elapsed: 240.9702939987

In [44]:
#write result.txt
start_time = time.time() # execution time
result = ""

for indo in indonesian_virus:
    for foreign in dict:
        virus_time = time.time()
        score = ""
        alignment = ""
        
        if len(foreign['seq']) > threshold:
            alignment = "Local"
            score = local_alignment(indo['seq'], foreign['seq'])
        else:
            alignment = "Global"
            score = global_alignment(indo['seq'], foreign['seq'])
        
        location = foreign['location']
        elapsed_time = time.time() - virus_time
        
        result += (indo['id'] + "|" + foreign['id'] + "|" + location + "|" + alignment + "|" + str(score) + "|" + str(elapsed_time) + "\n")

print("Total time elapsed: %s seconds" % (time.time() - start_time))
f = open('result.txt','w')
f.write(result)
f.close()

Total time elapsed: 2845.6476559638977 seconds


In [45]:
#read result.txt
f = open("result.txt","r")
lines = f.readlines()
f.close()

result = []

for line in lines:
    line_split = line.split('|')
    detail = {
        'indo'        : line_split[0],
        'foreign'     : line_split[1],
        'location'    : line_split[2],
        'alignment'   : line_split[3],
        'score'       : float(line_split[4]),
        'elapsed_time': float(line_split[5])
    }
    result.append(detail)

In [54]:
#split the virus
first_virus = result[:int(len(result)/2)]
second_virus = result[int(len(result)/2):]

In [55]:
#sort the result
sorted_first = sorted(first_virus, key=lambda x: x['score'], reverse=True)
sorted_second = sorted(second_virus, key=lambda x: x['score'], reverse=True)

In [56]:
sorted_first

[{'alignment': 'Local',
  'elapsed_time': 20.73088312149048,
  'foreign': 'KF993678.1',
  'indo': 'KU179098.1',
  'location': 'Canada',
  'score': 1132.0},
 {'alignment': 'Local',
  'elapsed_time': 22.11390495300293,
  'foreign': 'KU509998.3',
  'indo': 'KU179098.1',
  'location': 'Haiti',
  'score': 1130.0},
 {'alignment': 'Local',
  'elapsed_time': 22.34393882751465,
  'foreign': 'KX051563.1',
  'indo': 'KU179098.1',
  'location': 'USA',
  'score': 1130.0},
 {'alignment': 'Local',
  'elapsed_time': 22.06683921813965,
  'foreign': 'KJ776791.1',
  'indo': 'KU179098.1',
  'location': 'French Polynesia',
  'score': 1130.0},
 {'alignment': 'Local',
  'elapsed_time': 21.96515679359436,
  'foreign': 'KU647676.1',
  'indo': 'KU179098.1',
  'location': 'Martinique',
  'score': 1128.0},
 {'alignment': 'Local',
  'elapsed_time': 22.00878381729126,
  'foreign': 'KU497555.1',
  'indo': 'KU179098.1',
  'location': 'Brazil',
  'score': 1128.0},
 {'alignment': 'Local',
  'elapsed_time': 21.055214881

In [57]:
sorted_second

[{'alignment': 'Local',
  'elapsed_time': 7.22956109046936,
  'foreign': 'KF993678.1',
  'indo': 'KF258813.1',
  'location': 'Canada',
  'score': 392.0},
 {'alignment': 'Local',
  'elapsed_time': 7.226809978485107,
  'foreign': 'EU545988.1',
  'indo': 'KF258813.1',
  'location': 'Micronesia',
  'score': 392.0},
 {'alignment': 'Local',
  'elapsed_time': 7.529456853866577,
  'foreign': 'KX101064.1',
  'indo': 'KF258813.1',
  'location': 'Brazil',
  'score': 390.0},
 {'alignment': 'Local',
  'elapsed_time': 7.186153173446655,
  'foreign': 'KU866423.2',
  'indo': 'KF258813.1',
  'location': 'China',
  'score': 390.0},
 {'alignment': 'Local',
  'elapsed_time': 7.427290916442871,
  'foreign': 'KU647676.1',
  'indo': 'KF258813.1',
  'location': 'Martinique',
  'score': 390.0},
 {'alignment': 'Local',
  'elapsed_time': 7.573949098587036,
  'foreign': 'KX280026.1',
  'indo': 'KF258813.1',
  'location': 'Brazil',
  'score': 390.0},
 {'alignment': 'Local',
  'elapsed_time': 7.594936847686768,
  '